<a href="https://colab.research.google.com/github/abood-W/ai_python/blob/main/group_of_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install mistralai
!pip install langchain langchain-mistralai faiss-cpu


In [81]:
!pip install langchain langchain-mistralai faiss-cpu
!pip install langchain_community
!pip install -U langchain-tavily
!pip install chromadb

In [82]:
!pip install wikipedia

In [83]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from google.colab import userdata
import csv
import os

In [84]:
texts = [
    "Newton's second law states that Force equals Mass times Acceleration (F=ma).",
    "LangChain is a framework for building context-aware reasoning applications.",
    "Python was created by Guido van Rossum and first released in 1991."
]

In [107]:
def critic(task: str , result:str , success:bool):
    log_file = "agent_feedback.csv"
    file_exists = os.path.isfile(log_file)

    with open(log_file, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(["task", "result", "success"])
        writer.writerow([task, result, int(success)])
    return "Feedback recorded"

In [86]:
api_key = userdata.get('MISTRAL_API_KEY')
llm = ChatMistralAI(model="mistral-small",
                    temperature=0,
                    api_key=api_key
                    )

In [87]:
embeddings = MistralAIEmbeddings(api_key=api_key)

vectorstore = FAISS.from_texts(texts, embeddings)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

/usr/local/lib/python3.12/dist-packages/langchain_mistralai/embeddings.py:186: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [88]:
def math_tool(expr: str) -> str:
    return str(eval(expr))

In [89]:
"""tools = [
    Tool(name="MathTool", func=math_tool, description="Performs math calculations")
]"""

'tools = [\n    Tool(name="MathTool", func=math_tool, description="Performs math calculations")\n]'

In [90]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
wiki_api = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

**MEMORY**

In [91]:
from langchain.memory import ConversationBufferMemory

In [92]:
math_tools = [Tool(name="MathTool", func=lambda x: str(eval(x)), description="Do math calculations")]
math_agent = initialize_agent(
    math_tools,
    llm,
    agent="conversational-react-description", memory=ConversationBufferMemory(return_messages=True), verbose=True
    )

In [93]:
wiki_agent = initialize_agent(
    [wiki_tool],
    llm,
    agent="conversational-react-description",
    memory=ConversationBufferMemory(return_messages=True), verbose=True)


In [94]:
from langchain.chains import RetrievalQA

In [95]:
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
rag_agent = initialize_agent([Tool(name="RAGTool", func=rag_chain.run, description="Retrieve and answer from documents")],
                             llm, agent="conversational-react-description", memory=ConversationBufferMemory(return_messages=True), verbose=True)

In [96]:
tools = [
    Tool(name="MathExpert", func=math_agent.run, description="Expert in math calculations"),
    Tool(name="WikiExpert", func=wiki_agent.run, description="Eaxpert in Wikipedia knowledge"),
    Tool(name="RAGExpert", func=rag_agent.run, description="Expert in retrieving from custom documents"),
    Tool(name="Critic", func=lambda x: critic(x["task"], x["result"], x["success"]), description="Evaluate and log results")
]


In [97]:
main_memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", return_messages=True)

In [98]:
from langchain.agents import initialize_agent

main_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="conversational-react-description",
    memory=main_memory,
    verbose=True
)

In [99]:
response = main_agent.invoke({"input": "What's the capital of France?"})
print(response["output"])




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No.
AI: The capital of France is Paris.

> Finished chain.
The capital of France is Paris.


In [100]:
response = main_agent.invoke({"input": "What's the square root of 25?"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MathExpert
Action Input: sqrt(25)
Observation: The square root of 25 is 5.

AI: The square root of 25 is 5.

> Finished chain.
The square root of 25 is 5.


In [101]:
response = main_agent.invoke({"input": "Search about relativity theorem"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: WikiExpert
Action Input: Relativity theorem
Observation: The theory of relativity is a scientific theory proposed by Albert Einstein that describes the laws of physics in the presence of gravity. The theory of relativity is divided into two parts: special relativity and general relativity. Special relativity applies to all physical phenomena in the absence of gravity. It introduces two revolutionary concepts: that time is not absolute but can vary relative to an observer in motion, and that the speed of light in a vacuum is constant and is independent of the motion of the light source or the observer. General relativity, on the other hand, is a theory of gravitation that was developed by Einstein between 1907 and 1915. According to general relativity, the observed gravitational effect between masses results from their warping of spacetime.

Thought: Do I need to use a tool? No
AI: The theory of relati

In [102]:
response = main_agent.invoke({"input": "may name is mohammed"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello Mohammed, it's nice to meet you! Is there something I can help you with?

> Finished chain.
Hello Mohammed, it's nice to meet you! Is there something I can help you with?


In [103]:
response = main_agent.invoke({"input": "what is may name"})
print(response["output"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello! It seems there was a typo in your previous message. If you want to tell me your name is Mohammed, that's great! Nice to meet you, Mohammed. Is there something you'd like to know or discuss?

> Finished chain.
Hello! It seems there was a typo in your previous message. If you want to tell me your name is Mohammed, that's great! Nice to meet you, Mohammed. Is there something you'd like to know or discuss?


In [104]:
ans = main_agent.invoke("what is may name")
print(ans['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hello! It seems there was a typo in your previous message. If you want to tell me your name is Mohammed, that's great! Nice to meet you, Mohammed. Is there something you'd like to know or discuss?

> Finished chain.
Hello! It seems there was a typo in your previous message. If you want to tell me your name is Mohammed, that's great! Nice to meet you, Mohammed. Is there something you'd like to know or discuss?


In [121]:
def ask_with_feedback(agent, query, correct_answer,memory):

    ans = agent.run(query)

    success = str(correct_answer) in ans


    critic(task=query, result=ans, success=success)

    memory.chat_memory.add_user_message(query)

    memory.chat_memory.add_ai_message(ans)

    return ans


In [122]:
answer = ask_with_feedback(main_agent, "5 * 12", "60", main_memory)
print("Agent Answer:", answer)




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No

AI: The product of 5 and 12 is 60. I've noticed that you've asked for this calculation a few times. Is there something specific you would like to do with this calculation, or is there another question or topic you would like to discuss? I'm here to help!

> Finished chain.
Agent Answer: The product of 5 and 12 is 60. I've noticed that you've asked for this calculation a few times. Is there something specific you would like to do with this calculation, or is there another question or topic you would like to discuss? I'm here to help!


In [125]:
ans = ask_with_feedback(main_agent, "what is Newton second law", "F=ma", main_memory)
print("Agent Answer:", ans)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Newton's second law states that the force acting on an object is equal to the mass of that object times its acceleration. This is often expressed in the formula F = ma, where F is the force, m is the mass, and a is the acceleration. This law is fundamental in understanding how objects move and how forces affect them. Is there anything specific you would like to know about Newton's second law? I'm here to help!

> Finished chain.
Agent Answer: Newton's second law states that the force acting on an object is equal to the mass of that object times its acceleration. This is often expressed in the formula F = ma, where F is the force, m is the mass, and a is the acceleration. This law is fundamental in understanding how objects move and how forces affect them. Is there anything specific you would like to know about Newton's second law? I'm here to help!


In [123]:
import pandas as pd
df = pd.read_csv("agent_feedback.csv")
print(df)

     task                                             result  success
0  5 * 12  {'input': '5 * 12', 'chat_history': [HumanMess...        0
1  5 * 12                     The product of 5 and 12 is 60.        1
2  5 * 12  The product of 5 and 12 is 60. I've noticed th...        1


In [126]:
print(main_memory.load_memory_variables({})["chat_history"])

[HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's the square root of 25?", additional_kwargs={}, response_metadata={}), AIMessage(content='The square root of 25 is 5.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Search about relativity theorem', additional_kwargs={}, response_metadata={}), AIMessage(content='The theory of relativity is a scientific theory proposed by Albert Einstein that describes the laws of physics in the presence of gravity. It is divided into two parts: special relativity and general relativity. Special relativity applies to all physical phenomena in the absence of gravity and introduces the concepts that time is not absolute and that the speed of light is constant. General relativity is a theory of gravitation that describes the observed gravitational effect between 